# Getting started with Word2Vec in Gensim and making it work!

The idea behind Word2Vec is pretty simple. We are making and assumption that you can tell the meaning of a word by the company it keeps. This is analogous to the saying *show me your friends, and I'll tell who you are*. So if you have two words that have very similar neighbors (i.e. the usage context is about the same), then these words are probably quite similar in meaning or are at least highly related. For example, the words `shocked`,`appalled` and `astonished` are typically used in a similar context. 

In this tutorial, you will learn how to use the Gensim implementation of Word2Vec and actually get it to work! I have heard a lot of complaints about poor performance etc, but its really a combination of two things, (1) your input data and (2) your parameter settings. Note that the training algorithms in this package were ported from the [original Word2Vec implementation by Google](https://arxiv.org/pdf/1301.3781.pdf) and extended with additional functionality.

### Imports and logging

First, we start with our imports and get logging established:

In [1]:
# imports needed and set up logging
import gzip
import gensim 
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


C:\Users\Hamburg\Anaconda3\envs\tensorflow2\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Dataset 
Next, is our dataset. The secret to getting Word2Vec really working for you is to have lots and lots of text data. In this case I am going to use data from the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset. This dataset has full user reviews of cars and hotels. I have specifically concatenated all of the hotel reviews into one big file which is about 97MB compressed and 229MB uncompressed. We will use the compressed file for this tutorial. Each line in this file represents a hotel review. You can download the OpinRank Word2Vec dataset here.

To avoid confusion, while gensim’s word2vec tutorial says that you need to pass it a sequence of sentences as its input, you can always pass it a whole review as a sentence (i.e. a much larger size of text), and it should not make much of a difference. 

Now, let's take a closer look at this data below by printing the first line. You can see that this is a pretty hefty review.

In [3]:
data_file="reviews_data.txt.gz"

with gzip.open ('reviews_data.txt.gz', 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break


b"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in Be

### Read files into a list
Now that we've had a sneak peak of our dataset, we can read it into a list so that we can pass this on to the Word2Vec model. Notice in the code below, that I am directly reading the 
compressed file. I'm also doing a mild pre-processing of the reviews using `gensim.utils.simple_preprocess (line)`. This does some basic pre-processing such as tokenization, lowercasing, etc and returns back a list of tokens (words). Documentation of this pre-processing method can be found on the official [Gensim documentation site](https://radimrehurek.com/gensim/utils.html). 



In [4]:

def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_file))
    
    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info ("read {0} reviews".format (i))
            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess (line)

# read the tokenized reviews into a list
# each review item becomes a serries of words
# so this becomes a list of lists
documents = list (read_input (data_file))
logging.info ("Done reading data file")    

2018-02-19 12:19:56,429 : INFO : reading file reviews_data.txt.gz...this may take a while
2018-02-19 12:19:56,429 : INFO : read 0 reviews
2018-02-19 12:19:58,636 : INFO : read 10000 reviews
2018-02-19 12:20:00,853 : INFO : read 20000 reviews
2018-02-19 12:20:03,414 : INFO : read 30000 reviews
2018-02-19 12:20:05,773 : INFO : read 40000 reviews
2018-02-19 12:20:08,437 : INFO : read 50000 reviews
2018-02-19 12:20:11,057 : INFO : read 60000 reviews
2018-02-19 12:20:13,309 : INFO : read 70000 reviews
2018-02-19 12:20:15,688 : INFO : read 80000 reviews
2018-02-19 12:20:17,979 : INFO : read 90000 reviews
2018-02-19 12:20:20,453 : INFO : read 100000 reviews
2018-02-19 12:20:23,002 : INFO : read 110000 reviews
2018-02-19 12:20:25,636 : INFO : read 120000 reviews
2018-02-19 12:20:28,664 : INFO : read 130000 reviews
2018-02-19 12:20:31,621 : INFO : read 140000 reviews
2018-02-19 12:20:33,945 : INFO : read 150000 reviews
2018-02-19 12:20:35,920 : INFO : read 160000 reviews
2018-02-19 12:20:38,050

## Training the Word2Vec model

Training the model is fairly straightforward. You just instantiate Word2Vec and pass the reviews that we read in the previous step (the `documents`). So, we are essentially passing on a list of lists. Where each list within the main list contains a set of tokens from a user review. Word2Vec uses all these tokens to internally create a vocabulary. And by vocabulary, I mean a set of unique words.

After building the vocabulary, we just need to call `train(...)` to start training the Word2Vec model. Training on the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset takes about 10 minutes so please be patient while running your code on this dataset.

Behind the scenes we are actually training a simple neural network with a single hidden layer. But, we are actually not going to use the neural network after training. Instead, the goal is to learn the weights of the hidden layer. These weights are essentially the word vectors that we’re trying to learn. 

In [5]:
### PUT GENSIM CODE HERE
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

2018-02-19 12:21:02,991 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2018-02-19 12:21:02,992 : INFO : collecting all words and their counts
2018-02-19 12:21:02,993 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-02-19 12:21:03,294 : INFO : PROGRESS: at sentence #10000, processed 1655714 words, keeping 25777 word types
2018-02-19 12:21:03,559 : INFO : PROGRESS: at sentence #20000, processed 3317863 words, keeping 35016 word types
2018-02-19 12:21:03,913 : INFO : PROGRESS: at sentence #30000, processed 5264072 words, keeping 47518 word types
2018-02-19 12:21:04,276 : INFO : PROGRESS: at sentence #40000, processed 7081746 words, keeping 56675 word types
2018-02-19 12:21:04,661 : INFO : PROGRESS: at sentence #50000, processed 9089491 words, keeping 63744 word types
2018-02-19 12:21:05,046 : INFO : PROGRESS: at sentence #60000, processed 11013723 words, keeping 76781 word types
2018-02-19 12:21:05,329 : INFO : PROGRESS: a

2018-02-19 12:21:53,393 : INFO : PROGRESS: at 34.60% examples, 1308178 words/s, in_qsize 19, out_qsize 0
2018-02-19 12:21:54,395 : INFO : PROGRESS: at 35.50% examples, 1310248 words/s, in_qsize 20, out_qsize 3
2018-02-19 12:21:55,413 : INFO : PROGRESS: at 36.41% examples, 1312006 words/s, in_qsize 20, out_qsize 0
2018-02-19 12:21:56,431 : INFO : PROGRESS: at 37.24% examples, 1310555 words/s, in_qsize 17, out_qsize 2
2018-02-19 12:21:57,434 : INFO : PROGRESS: at 38.19% examples, 1310632 words/s, in_qsize 19, out_qsize 0
2018-02-19 12:21:58,437 : INFO : PROGRESS: at 39.05% examples, 1309285 words/s, in_qsize 19, out_qsize 0
2018-02-19 12:21:59,440 : INFO : PROGRESS: at 39.87% examples, 1306645 words/s, in_qsize 19, out_qsize 0
2018-02-19 12:22:00,454 : INFO : PROGRESS: at 40.68% examples, 1305190 words/s, in_qsize 18, out_qsize 1
2018-02-19 12:22:01,456 : INFO : PROGRESS: at 41.58% examples, 1306585 words/s, in_qsize 19, out_qsize 0
2018-02-19 12:22:02,461 : INFO : PROGRESS: at 42.34% ex

2018-02-19 12:23:05,625 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-02-19 12:23:05,625 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-02-19 12:23:05,641 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-02-19 12:23:05,641 : INFO : training on 207596775 raw words (151753870 effective words) took 112.5s, 1348765 effective words/s
2018-02-19 12:23:05,641 : INFO : training model with 10 workers on 70538 vocabulary and 150 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2018-02-19 12:23:06,658 : INFO : PROGRESS: at 0.38% examples, 1170736 words/s, in_qsize 20, out_qsize 0
2018-02-19 12:23:07,660 : INFO : PROGRESS: at 0.82% examples, 1259487 words/s, in_qsize 20, out_qsize 0
2018-02-19 12:23:08,663 : INFO : PROGRESS: at 1.20% examples, 1308754 words/s, in_qsize 16, out_qsize 3
2018-02-19 12:23:09,666 : INFO : PROGRESS: at 1.59% examples, 1314934 words/s, in_qsize 18, out_qsize 1
2018-02-19 12:23:10,684 : I

2018-02-19 12:24:20,124 : INFO : PROGRESS: at 31.90% examples, 1308491 words/s, in_qsize 19, out_qsize 0
2018-02-19 12:24:21,147 : INFO : PROGRESS: at 32.26% examples, 1308056 words/s, in_qsize 19, out_qsize 0
2018-02-19 12:24:22,155 : INFO : PROGRESS: at 32.57% examples, 1304716 words/s, in_qsize 18, out_qsize 1
2018-02-19 12:24:23,173 : INFO : PROGRESS: at 33.02% examples, 1303619 words/s, in_qsize 19, out_qsize 0
2018-02-19 12:24:24,184 : INFO : PROGRESS: at 33.51% examples, 1304852 words/s, in_qsize 18, out_qsize 1
2018-02-19 12:24:25,188 : INFO : PROGRESS: at 34.03% examples, 1306426 words/s, in_qsize 19, out_qsize 0
2018-02-19 12:24:26,198 : INFO : PROGRESS: at 34.55% examples, 1307658 words/s, in_qsize 18, out_qsize 1
2018-02-19 12:24:27,207 : INFO : PROGRESS: at 35.03% examples, 1308784 words/s, in_qsize 19, out_qsize 0
2018-02-19 12:24:28,218 : INFO : PROGRESS: at 35.51% examples, 1309971 words/s, in_qsize 19, out_qsize 0
2018-02-19 12:24:29,226 : INFO : PROGRESS: at 35.90% ex

2018-02-19 12:25:39,661 : INFO : PROGRESS: at 67.16% examples, 1324734 words/s, in_qsize 18, out_qsize 1
2018-02-19 12:25:40,671 : INFO : PROGRESS: at 67.62% examples, 1325068 words/s, in_qsize 19, out_qsize 0
2018-02-19 12:25:41,673 : INFO : PROGRESS: at 68.08% examples, 1325738 words/s, in_qsize 17, out_qsize 2
2018-02-19 12:25:42,674 : INFO : PROGRESS: at 68.53% examples, 1326172 words/s, in_qsize 19, out_qsize 0
2018-02-19 12:25:43,685 : INFO : PROGRESS: at 69.04% examples, 1326775 words/s, in_qsize 18, out_qsize 1
2018-02-19 12:25:44,690 : INFO : PROGRESS: at 69.53% examples, 1327330 words/s, in_qsize 19, out_qsize 0
2018-02-19 12:25:45,691 : INFO : PROGRESS: at 70.03% examples, 1327990 words/s, in_qsize 17, out_qsize 2
2018-02-19 12:25:46,697 : INFO : PROGRESS: at 70.50% examples, 1328739 words/s, in_qsize 19, out_qsize 0
2018-02-19 12:25:47,707 : INFO : PROGRESS: at 70.94% examples, 1329192 words/s, in_qsize 19, out_qsize 0
2018-02-19 12:25:48,702 : INFO : PROGRESS: at 71.30% ex

2018-02-19 12:26:52,001 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-02-19 12:26:52,009 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-02-19 12:26:52,010 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-02-19 12:26:52,011 : INFO : training on 415193550 raw words (303488696 effective words) took 226.4s, 1340731 effective words/s


303488696

## Now, let's look at some output 
This first example shows a simple case of looking up words similar to the word `dirty`. All we need to do here is to call the `most_similar` function and provide the word `dirty` as the positive example. This returns the top 10 similar words. 

In [5]:
w1 = "dirty"
model.most_similar(w1)



2018-02-15 20:08:25,026 : INFO : precomputing L2-norms of word weight vectors


[('filthy', 0.858426570892334),
 ('stained', 0.7780173420906067),
 ('unclean', 0.7708290815353394),
 ('grubby', 0.764405369758606),
 ('smelly', 0.7589772343635559),
 ('dusty', 0.7520752549171448),
 ('dingy', 0.7471007108688354),
 ('gross', 0.7170611023902893),
 ('grimy', 0.711336076259613),
 ('mouldy', 0.7099933624267578)]

That looks pretty good, right? Let's look at a few more. Let's look at similarity for `polite`, `france` and `shocked`. 

In [11]:
# look up top 6 words similar to 'polite'
w1 = ["polite"]
model.most_similar(w1, topn=6)


[('courteous', 0.9137427806854248),
 ('friendly', 0.8328721523284912),
 ('cordial', 0.7980687022209167),
 ('professional', 0.7894892692565918),
 ('attentive', 0.7774360775947571),
 ('personable', 0.7533614039421082)]

In [13]:
# look up top 6 words similar to 'france'
w1 = ["france"]
model.most_similar(w1, topn=6)

[('germany', 0.666722297668457),
 ('canada', 0.6492294669151306),
 ('spain', 0.6457390785217285),
 ('england', 0.6394527554512024),
 ('hawaii', 0.618595540523529),
 ('barcelona', 0.6143802404403687)]

In [14]:
# look up top 6 words similar to 'shocked'
w1 = ["shocked"]
model.most_similar(w1,topn=6)


[('horrified', 0.813444972038269),
 ('amazed', 0.7931602001190186),
 ('stunned', 0.7805903553962708),
 ('appalled', 0.7623282670974731),
 ('astonished', 0.7543476819992065),
 ('dismayed', 0.7507925033569336)]

That's, nice. You can even specify several positive examples to get things that are related in the provided context and provide negative examples to say what should not be considered as related. In the example below we are asking for all items that *relate to bed* only:

In [10]:
# get everything related to stuff on the bed
w1 = ["bed",'sheet','pillow']
w2 = ['couch']
model.most_similar(w1, w2)


[('blanket', 0.6916123032569885),
 ('duvet', 0.686753511428833),
 ('mattress', 0.6822925209999084),
 ('quilt', 0.6781111359596252),
 ('matress', 0.6581498384475708),
 ('sheets', 0.6393461227416992),
 ('pillows', 0.631115734577179),
 ('pillowcase', 0.623105525970459),
 ('foam', 0.6123258471488953),
 ('comforter', 0.5886681079864502)]

### Similarity between two words in the vocabulary

You can even use the Word2Vec model to return the similarity between two words that are present in the vocabulary. 

In [22]:
# similarity between two different words
model.similarity('london', 'paris')

0.3489625593721133

In [20]:
# similarity between two identical words
model.similarity('paris', 'paris')

1.0

In [23]:
# similarity between two unrelated words
model.similarity('bed', 'france')

-0.0969603706452369

Under the hood, the above three snippets computes the cosine similarity between the two specified words using word vectors of each. From the scores, it makes sense that `dirty` is highly similar to `smelly` but `dirty` is dissimilar to `clean`. If you do a similarity between two identical words, the score will be 1.0 as the range of the cosine similarity score will always be between [0.0-1.0]. You can read more about cosine similarity scoring [here](https://en.wikipedia.org/wiki/Cosine_similarity).

### Find the odd one out
You can even use Word2Vec to find odd items given a list of items.

In [26]:
# Which one is the odd one out in this list?
list = ["cat","dog","france"]
model.doesnt_match(list)

'france'

In [27]:
# Which one is the odd one out in this list?
list = ["bed","pillow","duvet","shower"]
model.doesnt_match(list)

'shower'

## Understanding some of the parameters
To train the model earlier, we had to set some parameters. Now, let's try to understand what some of them mean. For reference, this is the command that we used to train the model.

```
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
```

### `size`
The size of the dense vector to represent each token or word. If you have very limited data, then size should be a much smaller value. If you have lots of data, its good to experiment with various sizes. A value of 100-150 has worked well for me. 

### `window`
The maximum distance between the target word and its neighboring word. If your neighbor's position is greater than the maximum window width to the left and the right, then, some neighbors are not considered as being related to the target word. In theory, a smaller window should give you terms that are more related. If you have lots of data, then the window size should not matter too much, as long as its a decent sized window. 

### `min_count`
Minimium frequency count of words. The model would ignore words that do not statisfy the `min_count`. Extremely infrequent words are usually unimportant, so its best to get rid of those. Unless your dataset is really tiny, this does not really affect the model.

### `workers`
How many threads to use behind the scenes?


## When should you use Word2Vec?

There are many application scenarios for Word2Vec. Imagine if you need to build a sentiment lexicon. Training a Word2Vec model on large amounts of user reviews helps you achieve that. You have a lexicon for not just sentiment, but for most words in the vocabulary. 

Beyond, raw unstructured text data, you could also use Word2Vec for more structured data. For example, if you had tags for a million stackoverflow questions and answers, you could find tags that are related to a given tag and recommend the related ones for exploration. You can do this by treating each set of co-occuring tags as a "sentence" and train a Word2Vec model on this data. Granted, you still need a large number of examples to make it work. 
